In [1]:
!pip install azure-cosmos


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from azure.cosmos import CosmosClient, exceptions, PartitionKey
import json

In [3]:
# initialize cosmos client
endpoint = 'https://azure-cosmos-demo-nosql-noelrobert.documents.azure.com:443/'
key = '0cQSKDUeovU996kVJ5HmFaTgBfgiTFlj1KR4q1UdjSkQC2P9KTAobt08YEIy5j0QHth9AKz87fAlACDbFnuRcA=='
DATABASE_NAME = 'customer_database'
CONTAINER_NAME = 'customer_container'

In [4]:
# create a cosmos client
client = CosmosClient(endpoint, key)

# select database
database = client.get_database_client(DATABASE_NAME)

# select container name
container = database.get_container_client(CONTAINER_NAME)

In [9]:
with open('customer.json', 'r') as file:
    data = json.load(file)

print(data)
print(len(data))

[{'id': 'ust10004', 'name': 'neha', 'product': 'mouse', 'brand-name': 'sony', 'Country': 'United States', 'amount': 120000}, {'id': 'ust10005', 'name': 'vikram', 'product': 'smartphone', 'brand-name': 'hp', 'Country': 'Brazil', 'amount': 7000}, {'id': 'ust10006', 'name': 'rohini', 'product': 'tablet', 'brand-name': 'asus', 'Country': 'China', 'amount': 2000}, {'id': 'ust10007', 'name': 'mohit', 'product': 'headphones', 'brand-name': 'sony', 'Country': 'India', 'amount': 30000}, {'id': 'ust10008', 'name': 'vikram', 'product': 'keyboard', 'brand-name': 'lg', 'Country': 'India', 'amount': 150000}, {'id': 'ust10009', 'name': 'vikram', 'product': 'smartphone', 'brand-name': 'lg', 'Country': 'Canada', 'amount': 25000}, {'id': 'ust10010', 'name': 'neha', 'product': 'laptop', 'brand-name': 'logitech', 'Country': 'India', 'amount': 50}, {'id': 'ust10011', 'name': 'anjali', 'product': 'tablet', 'brand-name': 'samsung', 'Country': 'India', 'amount': 150000}, {'id': 'ust10012', 'name': 'anjali', '

In [10]:
# Q1: insert records into Azure CosmosDB from JSON file
for item in data:
    try:
        container.create_item(item)
    
    except exceptions.CosmosHttpResponseError as e:
        print(f'An error has occurred: {e.message}')

In [28]:
# Q2.1 update records
unique_id = 'ust10007'
new_brand_name = 'Sony-Ericsson'

try:
    items = container.query_items(query=f"""SELECT * FROM c WHERE c.id='{unique_id}'""", enable_cross_partition_query=True)

    for item in items:
        item['brand-name'] = new_brand_name
        container.upsert_item(item) # alternative, replace_item only replaces and throuws error is its not already present
        # [for same id, if we try changing partition key, it will be a new record - inorder to handle this properly, use replace_item()]



except exceptions.CosmosHttpResponseError as e:
    print(f'An error has occurred: {e.message}')

In [21]:
# Q2.2 delete records
unique_id = 'ust10004'
product_partition_key_value = 'mouse'

for item in container.query_items(query=f"""SELECT * FROM c WHERE c.id='{unique_id}'""", enable_cross_partition_query=True):
    container.delete_item(item, partition_key=product_partition_key_value)
    # print(item)

In [17]:
# Q3: write data from customer_container to bank.json

output_file = 'bank.json'
try:
    items = container.query_items(
        query="""SELECT * FROM c""",
        enable_cross_partition_query=True # needed when data partitioned by multiple keys
    )

    itemlist = list(items)   
    with open(output_file, 'w') as file:
        json.dump(itemlist, file, indent=4)  

except exceptions.CosmosHttpResponseError as e:
    print(f'An error has occurred: {e.message}')